In [8]:
import os
import sys
import json
import re

from glob import glob
from logging import error, warning

from common import FINNISH_MAIN_REGISTER, CLASS_ABBREV_MAP


def get_docid_class_map():
    docid_class_map = {}
    with open('../data/docid-label-map.tsv') as f:
        for ln, l in enumerate(f, start=1):
            l = l.rstrip()
            fields = l.split('\t')
            docid, class_ = fields
            if docid in docid_class_map:
                raise ValueError('duplicate docid: {}'.format(docid))
            docid_class_map[docid] = class_
    return docid_class_map


def load_biarc_data(dirpath):
    docid_class_map = get_docid_class_map()
    biarcs, classes = [], []
    for fn in glob('{}/*.txt'.format(dirpath)):
        m = re.match(r'^(.*?)-barcs\.', os.path.basename(fn))
        docid = m.group(1)
        class_ = docid_class_map[docid]
        with open(fn) as f:
            data = f.read().strip().split('\t')
        class_ = FINNISH_MAIN_REGISTER[class_]
        if class_ is None:
            continue    # class doesn't map across languages
        biarcs.append(data)
        classes.append(CLASS_ABBREV_MAP[class_])
    return biarcs, classes


train_biarcs, train_classes = load_biarc_data('../data/split-biarcs/train/')
devel_biarcs, devel_classes = load_biarc_data('../data/split-biarcs/dev/')

In [9]:
from collections import Counter


MIN_EXAMPLES = 25    # filter classes with fewer


class_count = Counter()
for c in train_classes:
    class_count[c] += 1
target_class = set(c for c, v in class_count.items() if v >= MIN_EXAMPLES)


def filter_by_class(data, classes, targets):
    filtered_data, filtered_classes = [], []
    for t, c in zip(data, classes):
        if c in targets:
            filtered_data.append(t)
            filtered_classes.append(c)
    return filtered_data, filtered_classes


train_biarcs, train_classes = filter_by_class(train_biarcs, train_classes, target_class)
devel_biarcs, devel_classes = filter_by_class(devel_biarcs, devel_classes, target_class)

In [10]:
from pprint import pprint

def class_counts(classes):
    counter = Counter()
    for c in classes:
        counter[c] += 1
    return counter


pprint(class_counts(train_classes))

Counter({'Narrative': 422,
         'Info-Persuasion': 269,
         'D-Informational': 218,
         'Opinion': 199,
         'How-to': 86,
         'Discussion': 84})


In [11]:
def biarcs_to_text(biarcs):
    return ['\t'.join(b) for b in biarcs]


train_texts = biarcs_to_text(train_biarcs)
devel_texts = biarcs_to_text(devel_biarcs)

In [12]:
from itertools import combinations

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC


def biarc_tokenizer(text):
    for biarc in text.split('\t'):
        yield biarc
        parts = biarc.split(' ')
        for twoparts in combinations(parts, 2):
            yield '---'.join(twoparts)
        for part in parts:
            yield part


# detailed classes: 0.5582

#vectorizer = CountVectorizer(analyzer='word', tokenizer=biarc_tokenizer, lowercase=False, ngram_range=(1,1))
vectorizer = TfidfVectorizer(analyzer='word', tokenizer=biarc_tokenizer, lowercase=False, ngram_range=(1,1))
vectorizer.fit(train_texts)

train_X = vectorizer.transform(train_texts)
devel_X = vectorizer.transform(devel_texts)

classifier = LinearSVC(C=1)
classifier.fit(train_X, train_classes)

classifier.score(devel_X, devel_classes)

0.5248618784530387

In [13]:
import eli5


eli5.show_weights(classifier, vec=vectorizer, top=100)

In [15]:
import numpy as np

from sklearn.metrics import confusion_matrix
from pandas import DataFrame


pred_Y = classifier.predict(devel_X)
cm = confusion_matrix(devel_classes, pred_Y)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]    # normalize
df = DataFrame(cm * 100, index=classifier.classes_, columns=classifier.classes_)
df.round(2)

,D-Informational,Discussion,How-to,Info-Persuasion,Narrative,Opinion
D-Informational,22.58,3.23,0.00,32.26,29.03,12.90
Discussion,0.00,9.09,0.00,0.00,63.64,27.27
How-to,8.33,0.00,16.67,25.00,41.67,8.33
Info-Persuasion,5.41,0.00,0.00,54.05,35.14,5.41
Narrative,8.20,0.00,0.00,6.56,73.77,11.48
Opinion,0.00,0.00,0.00,10.34,20.69,68.97
